In [57]:
import re
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

## Extracting data

In [220]:
%%time
df = pd.read_csv('chess_games.csv', nrows=500000,
                usecols=['Event', 'Result', 'UTCDate', 'Opening', 'Termination', 'AN']) # chunksize=100000, index_col=''
print(f"{len(df)} rows was extracted")

500000 rows was extracted
CPU times: total: 4.98 s
Wall time: 7.46 s


In [221]:
df.head(3)

,Event,Result,UTCDate,Opening,Termination,AN
0,Classical,1-0,2016.06.30,Slav Defense,Time forfeit,1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1,Blitz,0-1,2016.06.30,King's Pawn Opening: 2.b3,Normal,1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2,Blitz tournament,1-0,2016.06.30,Scandinavian Defense: Mieses-Kotroc Variation,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....


## Transform data

### Filter dataframe

#### Get rid of ambiguous match results and matches that was abandoned or have rules infractions

In [222]:
filt = (df['Result'] != '*') & (df['Termination'] != 'Abandoned') & (df['Termination'] != 'Rules infraction')
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

1057 rows was removed
498943 rows left


#### Remove short matches (~less than 10 moves)

In [223]:
filt = df["AN"].apply(len) > 50
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

5689 rows was removed
493254 rows left


#### Get rid of unstructured string values in "AN" column

In [224]:
filt = ~df['AN'].str.contains('\[%eval')
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

63866 rows was removed
429388 rows left


#### Remove unpopular openings (less than 1500 matches)

In [179]:
# value_counts = df.stack().value_counts() # Entire DataFrame 
# to_remove = value_counts[value_counts <= 1500].index
# df.replace(to_remove, np.nan, inplace=True)

In [84]:
vc = df['Opening'].value_counts()
vals_to_remove = vc[vc < 1500].index.values
df['Opening'].loc[df['Opening'].isin(vals_to_remove)] = '111'
filt = df['Opening'] != '111'
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

353753 rows was removed
505481 rows left


### Expand and rearrange dataframe

#### Reset index

In [225]:
df = df.reset_index(drop=True)

#### Create column "Tournament"

In [226]:
df['Tournament'] = df['Event'].str.contains('tournament')

#### Rearrange column "Event"

In [227]:
df['Event'] = df['Event'].map({' Classical ': 'Classical',
                               ' Blitz ': 'Blitz',
                               ' Blitz tournament ': 'Blitz',
                               ' Correspondence ': 'Classical',
                               ' Classical tournament ': 'Classical',
                               ' Bullet tournament ': 'Bullet',
                               ' Bullet ': 'Bullet',
                               'Blitz tournament ': 'Blitz',
                               'Bullet ': 'Bullet',
                               'Classical ': 'Classical',
                               'Blitz ': 'Blitz'
                               })

#### Change columns datatype

In [228]:
df['Event'] = df['Event'].astype('category')
df['Result'] = df['Result'].astype('category')
df['Termination'] = df['Termination'].astype('category')
df['UTCDate'] = pd.to_datetime(df['UTCDate'])

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429388 entries, 0 to 429387
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Event        429388 non-null  category      
 1   Result       429388 non-null  category      
 2   UTCDate      429388 non-null  datetime64[ns]
 3   Opening      429388 non-null  object        
 4   Termination  429388 non-null  category      
 5   AN           429388 non-null  object        
 6   Tournament   429388 non-null  bool          
dtypes: bool(1), category(3), datetime64[ns](1), object(2)
memory usage: 11.5+ MB


#### Create moves_total from AN column

In [231]:
def delete_result_from_an(move: str) -> str:
    '''Delete result mark in the last move.'''
    move = re.sub(' 1-0',     '', move)
    move = re.sub(' 0-1',     '', move)
    move = re.sub(' 1/2-1/2', '', move)
    move = re.sub(' \*',      '', move)
    return move

In [232]:
def populate_moves_total(row: list[str]) -> None:
    '''
    Split each move (string) into a tuple.
    Tuple format: (match_id, move_num, white_move, black_move)
    Match_id is foright key of the main dataframe (index column).
    '''
    for move in row:
        split_list = move.split(' ')
        tuple_move = tuple([0] + [int(split_list[0])] + split_list[1:])

        if len(tuple_move) == 3:
            tuple_move = tuple_move + (None, ) # in case of last move ('0', '38', 'Rfd1', None)

        moves_total.append(tuple_move)

In [233]:
def create_moves_total(row: str) -> None:
    '''Separate each move.'''
    match_moves = []
    match_moves.append(row)
    sep_num = 2
    while True:  # parse as many moves as possible
        separator = str(sep_num)+'.'
        if separator in match_moves[-1]:  # if the next move is found
            match_moves = match_moves[0:-1] + match_moves[-1].split(' '+separator) # concatenate all previous moves and remainder
            match_moves[-1] = str(sep_num) + match_moves[-1]
            sep_num+=1
        else:  # if the next move is not found
            match_moves[-1] = delete_result_from_an(match_moves[-1])
            match_moves[0] =  re.sub('1.', '1', match_moves[0])
            break
    
    populate_moves_total(match_moves)

In [234]:
%%time
%%capture
moves_total: list[tuple] = [] # template for data frame
df["AN"].apply(create_moves_total)

CPU times: total: 57.9 s
Wall time: 59.2 s


#### Convert moves_total to df_moves

In [235]:
%%time
%%capture
df_moves = pd.DataFrame(moves_total, columns=['match_id', 'move_num', 'white_move', 'black_move'])

CPU times: total: 12 s
Wall time: 59.6 s


## Draft

In [125]:
df_moves['move_num'].max()

290

In [129]:
df_moves.query('move_num >= 290')

,match_id,move_num,white_move,black_move
13634638,0,290,Kf2,None


In [ ]:
# %%time
# for i, row in df.iterrows():
#     df.at[i, 'Event'] = df.at[i, 'Event']

In [ ]:
# %%time
# for i in df.index:
#     df['Event'].iloc[i] = df['Event'].iloc[i]

In [188]:
length = 0
for row in range(len(df)):
    length = length + len(df['AN'][row])
length

35474522